
# Lab 3: Using Mirror's Featuers
## Using Mirror feature on volumes for disaster recovery or development use.

### With Data Fabric Mirroring you can:

- Mirror data from one cluster to another for disaster recovery (not showcased here today)
- Mirror data localy on one cluster for recovery, development, testing or data validation needs.
- Schedules mirrors or run them on demand.
- Promote mirrors out of read-only mode to read-write

<p align="center">
  <img src="Pictures/edf-mirror-overview.png" width="600" height="400" alt="Data Fabric Snapshots">                                                    
  
</p>

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Data-Fabric101'] }}

echo "We are now connected to the appliance"

#
# environment variables to be adjusted/verified by the student
#these are ansible variables that are being replaced at the time of the deployment
username="student{{ STDID }}" # your Data Fabric login credentials - username  
password="{{ PASSSTU }}" # your Data Fabric login credentials - password - 
apiserver_host="{{ DATAFABRICAPISERVERIP }}" # API Server for the Data Fabric Cluster
ad_group="{{ DATAFABRICADGROUP }}" # LDAP Domain Group for Data Fabric users
vol_name="{{ DATAFABRICVOL }}"   # suffix for student volume name
src-mirrvol1="{{ DATAFABRICVOLSRCVOLNAME }}" # suffix for student source mirror volume name
dst-mirrvol1="{{ DATAFABRICVOLDSTVOLNAME }}"  # suffix for student destination mirror volume name
cluster_name="{{ DATAFABRICCLUSTERAME }}"  # Data Fabric Cluster Name
#
# fixed environment variables setup by the HPE Data Fabrics Lab administrator - please DO NOT MODIFY!!
#these are ansible variables that are being replaced at the time of the deplaoyment

echo "your operation context is:" student{{ STDID }} "with Password:" {{ PASSSTU }}

## 1. Creating a source volume that we will use to mirror from.

#### Background:

You have been given a user that is allowed to create and modify volumes on the cluster.  Data Fabric volumes can be mounted so they are accessable from NFS, HDFS or POSIX commands (like the bash kernel shell in these labs).  For security reasons, you can only mount volumes in the /user directory.

Fist let's create a volume to create mirror's on:

In [ ]:
curl -k -X POST 'https://{{ DATAFABRICAPISERVERIP }}:8443/rest/volume/create?name=student{{ STDID }}-{{ DATAFABRICVOLSRCVOLNAME }}&path=/user/student{{ STDID }}-{{ DATAFABRICVOLSRCVOLNAME }}' --user student{{ STDID }}:{{ PASSSTU }}

## 2. Create a mirror volume from your source volume.

Now let's create a local mirror volume from our source volume we made in the previous step:

In [ ]:
curl -k -X POST 'https://{{ DATAFABRICAPISERVERIP }}:8443/rest/volume/create?name=student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}&path=/user/student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}&type=mirror&source=student{{ STDID }}-$src_vol_name@{{ DATAFABRICCLUSTERAME }}' --user student{{ STDID }}:{{ PASSSTU }}

## 3. Create data within your source volume

Now let's create some data in our source volume so we can verify that the mirror operation worked:

In [ ]:
echo "student{{ STDID }}-{{ DATAFABRICVOL }} source volume update $(date)" >> /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICVOLSRCVOLNAME }}/student{{ STDID }}-source-volume-file1
ls /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICVOLSRCVOLNAME }}
cat /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICVOLSRCVOLNAME }}/student{{ STDID }}-source-volume-file1

## 4. Start the mirroring process

By default a target or destination mirror volume is mounted as read-only (ro).  Also by default the mirror is not started until you manually start it or apply a schedule to the mirror.  In this example we will be starting it manually.

In [ ]:
curl -k -X POST 'https://{{ DATAFABRICAPISERVERIP }}:8443/rest/volume/mirror/start?name=student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}' --user student{{ STDID }}:{{ PASSSTU }}

This lab is a short term setup of managing data so we will not cover setting up schedules and applying them to snapshot's or to mirrors.  For more details on creating schedules see: [Creating Schedules on Ezmeral Data Fabric](https://docs.datafabric.hpe.com/62/ReferenceGuide/schedule.html)

## 5. Verify your source data is now within your destination volume

Now let's verify the data we wrote in our previous steps in our source volume is now in the destination volume and that our mirror operation we started worked:

In [ ]:
ls /mapr/{{ DATAFABRICCLUSTERAME }}e/user/student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}

Now let's write some data to your new destination volume that was mirrored from the source volume:


In [ ]:
echo "student{{ STDID }}-{{ DATAFABRICVOL }} destination volume update $(date)" >> /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}'-'{{ DATAFABRICVOLDSTVOLNAME }}/student{{ STDID }}'-'source-volume-file1
ls /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}
cat /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}/student{{ STDID }}-source-volume-file1

Notice the error: "Read-only file system"
This error is to be expected as by default destination volumes that are currently being mirrored are mounted in read-only mode (ro).
Let's break the mirror and mount the volume in read-write (rw) mode:

## 6. Promoting destinaation volumes to read-write

First let's ensure the mirror process has stoped:

In [ ]:
curl -k -X POST 'https://{{ DATAFABRICAPISERVERIP }}:8443/rest/volume/mirror/stop?name=student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}' --user student{{ STDID }}:{{ PASSSTU }}

Now we can change the volume mount to read-write:

In [ ]:
curl -k -X POST 'https://{{ DATAFABRICAPISERVERIP }}:8443/rest/volume/modify?name=student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}&type=rw' --user student{{ STDID }}:{{ PASSSTU }}

Finally we can verify that the new destination volume has been mounted read-write (rw) by writing new data to it:

In [ ]:
echo "student{{ STDID }}-{{ DATAFABRICVOL }} destination volume update $(date)" >> /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}/student{{ STDID }}-source-volume-file1
ls /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}
cat /mapr/{{ DATAFABRICCLUSTERAME }}/user/student{{ STDID }}-{{ DATAFABRICVOLDSTVOLNAME }}/student{{ STDID }}-source-volume-file1

For additional information on using HPE Ezmeral Data Fabric Mirror's Features see: [Mirror Volumes](https://docs.datafabric.hpe.com/62/AdministratorGuide/MirrorVolumes-Mirroring.html)

## Mirror's Summary

In this example we:

1. Created a new source volume
2. Wrote data to it
3. Setup a new destination mirror volume
4. Started the mirror process on the destination volume.
5. And then promoted that mirror to be it's own volume and wrote new data to it.

# Using Access Security Settings and Policies  on your data
You can now move to the conclusion of this workshop !

* [Lab 4 - Using Access Security Settings and Policies](4-WKSHP-App-Security-Policies.ipynb)